In [1]:
include("../src/GaussianFilters.jl")
using .GaussianFilters # . done before adding to package repo

using LinearAlgebra
using Distributions
using Random

Random.seed!(1);

2D Example motion example where you provide x and y force directly through an object; u = \[Fx, Fy\].

As such a = F/m, discrete dV = adt = F/mdt. Let state vector x consist of \[px, vx, py, vy\]

In [2]:
dt = 0.1
m = 50
A = [1 dt 0 0 ; 0 1 0 0 ; 0 0 1 dt; 0 0 0 1]
B = [0 0; dt/m 0; 0 0; 0 dt/m]
W = 2*Matrix{Float64}(I,4,4)
dmodel = LinearDynamicsModel(A,B,W)

LinearDynamicsModel{Float64,Float64,Float64}([1.0 0.1 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.1; 0.0 0.0 0.0 1.0], [0.0 0.0; 0.002 0.0; 0.0 0.0; 0.0 0.002], [2.0 0.0 0.0 0.0; 0.0 2.0 0.0 0.0; 0.0 0.0 2.0 0.0; 0.0 0.0 0.0 2.0])

Assume we can noisily measure velocity

In [3]:
C = [0 1.0 0 0; 0 0 0 1.0]
V = 2*Matrix{Float64}(I,2,2)
omodel = LinearObservationModel(C,V)
kf = KalmanFilter(dmodel,omodel)

KalmanFilter(LinearDynamicsModel{Float64,Float64,Float64}([1.0 0.1 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.1; 0.0 0.0 0.0 1.0], [0.0 0.0; 0.002 0.0; 0.0 0.0; 0.0 0.002], [2.0 0.0 0.0 0.0; 0.0 2.0 0.0 0.0; 0.0 0.0 2.0 0.0; 0.0 0.0 0.0 2.0]), LinearObservationModel{Float64,Bool,Float64}([0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0], Bool[false false; false false], [2.0 0.0; 0.0 2.0]))

In [4]:
kf.d.W

4×4 Symmetric{Float64,Array{Float64,2}}:
 2.0  0.0  0.0  0.0
 0.0  2.0  0.0  0.0
 0.0  0.0  2.0  0.0
 0.0  0.0  0.0  2.0

Initialize belief, generate action sequence (forces in a circle), simulate model 

In [5]:
times = 0:dt:10
Fmag = 100
action_sequence = [[Fmag*cos(t), Fmag*sin(t)] for t in times]
b0 = GaussianBelief([0.0,0.0,0.0,0.0], 2.0*Matrix{Float64}(I,4,4))

sim_states, sim_measurements = simulation(b0,action_sequence,kf)

(Array{Float64,1}[[0.420429, 0.54079, -0.845183, -0.0147718], [-0.712056, 1.18077, 2.39908, -3.22092], [0.231509, 2.74204, 2.72582, -3.93965], [-0.475261, 0.429474, 2.50265, -4.97137], [0.515297, 1.39956, 1.91588, -3.02154], [-1.07016, 1.50857, 1.38019, -5.93524], [-0.116465, 2.76384, 1.32209, -6.75237], [-1.66307, 2.45476, 0.746507, -6.15607], [-1.12575, 1.18099, -1.07126, -4.43], [-1.08635, 2.39427, -1.55706, -4.79718]  …  [-5.78496, -7.4135, -54.1751, -3.693], [-6.15876, -6.94901, -55.0987, -5.47124], [-7.74577, -6.23461, -54.0395, -4.26124], [-8.98974, -7.14771, -54.4639, -6.22198], [-9.26235, -7.82595, -54.834, -6.22635], [-12.2017, -10.0606, -55.581, -6.59939], [-14.9616, -12.0046, -55.991, -8.09604], [-17.6645, -12.446, -57.9026, -7.88497], [-19.2891, -9.39122, -58.6928, -6.7419], [-21.5876, -10.21, -58.0268, -6.3973]], Any[[-0.712056, 1.18077, 2.39908, -3.22092], [0.231509, 2.74204, 2.72582, -3.93965], [-0.475261, 0.429474, 2.50265, -4.97137], [0.515297, 1.39956, 1.91588, -3.02

### Plot

In [ ]:
using PyPlot

figure()
## plot truths
# aircraft carrier
x = [k[1] for k in x_ac_truth]
y = [k[2] for k in x_ac_truth]
plot(x,y,label="Aircraft Carrier Truth","b")

# airplane 1
x = [k[1] for k in x_ap1_truth]
y = [k[2] for k in x_ap1_truth]
plot(x,y,label="Airplane Truths","r")

# airplane 2
x = [k[1] for k in x_ap2_truth]
y = [k[2] for k in x_ap2_truth]
plot(x,y,"r")

# airplane 3
x = [k[1] for k in x_ap3_truth]
y = [k[2] for k in x_ap3_truth]
plot(x,y,"r")

# airplane 4
x = [k[1] for k in x_ap4_truth]
y = [k[2] for k in x_ap4_truth]
plot(x,y,"r")

## plot estimates
for x in x_est
    for mu in x
        plot(mu[1],mu[2],"k.")
    end
end

## Formatting
title("Aircraft Carrier Example")
xlabel("x (m)")
ylabel("y (m)")

plot([],[],label="Estimated","k.")

legend()
xlim(-50,50)
ylim(-50,50)
show()